In [1]:
import sys
import os

In [2]:
import matplotlib.pyplot as plt
from ipywidgets import interactive, IntSlider

import numpy as np
import pandas as pd
import sqlite3
import torch

In [4]:
sys.path.append(os.path.join(sys.path[0], '../..'))

from endure.data.io import Reader
from endure.lsm.cost import EndureCost
from endure.lsm.types import Policy, System, LSMDesign, LSMBounds, Workload
from endure.ltune.util import LTuneEvalUtil
from endure.ltune.model import LTuneModelBuilder

In [5]:
connection = sqlite3.connect("../../../mlos_030924.db")
cursor = connection.cursor()

In [6]:
env_table = pd.read_sql("SELECT * FROM environments;", connection, index_col="env_id")
run_table = pd.read_sql("SELECT * FROM tunings;", connection, index_col="idx")

In [7]:
path = '/scratchNVMe/ndhuynh/other_data/models/ltune/klsm_med2'
config = Reader.read_config(os.path.join(path, 'endure.toml'))
cf = EndureCost(config)
design_type = getattr(Policy, config["lsm"]["design"])
bounds = LSMBounds(**config["lsm"]["bounds"])

In [16]:
model = LTuneModelBuilder(
    size_ratio_range=bounds.size_ratio_range,
    max_levels=bounds.max_considered_levels,
    **config["ltune"]["model"],
).build_model(design_type)
model_name = os.path.join(path, 'best.model')
# model_name = os.path.join(path, 'checkpoints/epoch_20.checkpoint')
model_data = torch.load(model_name)
model_params = model_data['model_state_dict'] if model_name.endswith('checkpoint') else model_data
status = model.load_state_dict(model_params)
model.eval()
status

<All keys matched successfully>

In [17]:
ltune_util = LTuneEvalUtil(config, model, design_type)

In [18]:
def plot_env(env_id: int = 1):
    env = env_table.query(f'env_id == {env_id}')
    data = run_table.query(f'env_id == {env_id}')
    wl_cols = ['empty_reads', 'non_empty_reads', 'range_queries', 'writes']
    sys_cols = ['entry_size', 'selectivity', 'entries_per_page', 'num_elmement', 'bits_per_elem_max', 'read_write_asym']
    wl = Workload(*env[wl_cols].values[0])
    system = System(*env[sys_cols].values[0])
    
    out = ltune_util.get_ltune_out(system, wl.z0, wl.z1, wl.q, wl.w)
    ltune_design = ltune_util.convert_ltune_output(out)
    ltune_cost = cf.calc_cost(ltune_design, system, wl.z0, wl.z1, wl.q, wl.w)
    
    x, y = data['round'].values, data['cost'].values
    aggregated_cost = []
    tmp = []
    for i in range(1, len(data['cost'].values)):
        aggregated_cost.append(sum(data['cost'].values[:i]))
    aggregated_cost.append(data['cost'].values[i]+aggregated_cost[i-1])

    ltune_aggregated_cost = ltune_cost * np.arange(1, len(x) + 1)
    fig, ax = plt.subplots()
    ax.plot(x, aggregated_cost, label='Bayesian Optimizer (Aggregated)', color='tab:gray')
    ax.plot(x, ltune_aggregated_cost, label='LTune (Aggregated)', color='tab:green', linestyle='--')
    ax.legend()
    ax.set_ylabel('Aggregated Cost (Avg I/O per Query)')
    ax.set_xlabel('Rounds')
    ax.set_title(f'Workload(z0={wl.z0:.2f}, z1={wl.z1:.2f}, q={wl.q:.2f}, w={wl.w:.2f})')
    plt.show()

    return env_id, ltune_design, wl, system

In [19]:
id_widget = IntSlider(min=1, max=100, step=1)
iplot = interactive(plot_env, env_id=id_widget)
iplot

interactive(children=(IntSlider(value=1, description='env_id', min=1), Output()), _dom_classes=('widget-intera…

In [15]:
iplot.result

(1,
 LSMDesign(h=0.4215482473373413, T=27, policy=<Policy.KHybrid: 3>, Q=1.0, Y=1.0, Z=1.0, K=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 Workload(z0=0.325, z1=0.008000000000000007, q=0.603, w=0.06399999999999995),
 System(E=8192.0, s=7.273692678982164e-09, B=4.0, N=954120869.0, H=13.452259319538351, phi=1.0))